In [4]:
class AccountActive():
    def __init__(self, func):
        self.func = func
        self.test = "Das ist ein Test"

    def __call__(self, *args, **kwargs):
        self.apply()
        return self.func(*args, **kwargs)
        
    def apply(self):
        print("Account Active - apply")
        


In [17]:
# WIth function wraper:
from functools import wraps

# This is working
class DecoratorClass():
    def __init__(self):
        self.decorator_attribute = True
        
    # pass the decorator instance and the decoratpr parameters here
    def __call__(self, decorator_param):
        print("__call__")
        
        # pass the decorated function here
        def decorator(func):
            print("__call__ + decorator")
            
            # pass the instance of the decorated bound method here and all general method parameters
            @wraps(func)
            def wrapper(instance, *args, **kwargs):
                print("method instance attribute =", instance.att)
                print("decorator parameter function call =", decorator_param)
                
                # execute decorator method
                self.decorator_method()
                
                # pass instance back to method and optionally the decorator instance along with the method parameters
                result = func(instance, self, *args, **kwargs)
                
                print("decorator attribute post function call =", self.decorator_attribute)
                return result
            return wrapper
        return decorator
    
    def decorator_method(self):
        print("Execute decorator method")
        
decorator = DecoratorClass() 

class externalClass():
    def __init__(self):
        self.att = True
    
    @decorator(True)
    def method(self, decorator_instance, method_parameter):
        print("access decorator attribute in method:", decorator_instance.decorator_attribute)
        print("method parameter:", method_parameter)
        pass
    
    

instance = externalClass()
instance.method(5)

__call__
__call__ + decorator
method instance attribute = True
decorator parameter function call = True
Execute decorator method
access decorator attribute in method: True
method parameter: 5
decorator attribute post function call = True


In [23]:
# With function wrapper and singleton:
from functools import wraps

class AccountActive():
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance.test = "Das ist ein Test"
            cls._instance.funcs = []
        return cls._instance
    
    def __call__(self, decorator_param):
        print("__call__")
        def decorator(func):
            print("__call__ + decorator")
            @wraps(func)
            def wrapper(instance, *args, **kwargs):            
                self.decorator_method(decorator_param)
                result = func(instance, self, *args, **kwargs)
                return result
            return wrapper
        return decorator
    
    def decorator_method(self, test):
        print("Execute decorator method: ", self.test, "parameter:", test)
        
accoutActive1 = AccountActive()
accoutActive2 = AccountActive()
        
class Vehicles():
    def __init__(self):
        self.type = "car"
    
    @accoutActive1("param1")
    def get1(self, decorator, type):
        print("Vehicles - get1: ", type)
        decorator.decorator_method("new param1")
        
    @accoutActive1("param2")    
    def get2(self, decorator, type):
        print("Vehicles - get2: ", type)
        decorator.decorator_method("new param2")
    
    @accoutActive2("param3")
    def get3(self, decorator, type):
        print("Vehicles - get3: ", type)
        decorator.decorator_method("new param3")
        

vehicle = Vehicles()
vehicle.get1("vw")
vehicle.get2("bmw")
vehicle.get3("audi")


__call__
__call__ + decorator
__call__
__call__ + decorator
__call__
__call__ + decorator
Execute decorator method:  Das ist ein Test parameter: param1
Vehicles - get1:  vw
Execute decorator method:  Das ist ein Test parameter: new param1
Execute decorator method:  Das ist ein Test parameter: param2
Vehicles - get2:  bmw
Execute decorator method:  Das ist ein Test parameter: new param2
Execute decorator method:  Das ist ein Test parameter: param3
Vehicles - get3:  audi
Execute decorator method:  Das ist ein Test parameter: new param3


In [33]:
# Using Class Descriptor without parameters

from functools import partial

class AccountActive():
    def __init__(self, func):
        print("__init__")
        self.func = func
        self.test = "Das ist ein Test"
        
    def __call__(self, *args, **kwargs):
        print("__call__")
        self.apply()
        return self.func(*args, **kwargs)
    
    def __get__(self, instance, owner = None):
        print("__get__")
        return partial(self.__call__, instance)
    
    def apply(self):
        print("Accout Active - apply")
        



class Vehicles():
    def __init__(self):
        self.type = "car"
    
    @AccountActive
    def get(self):
        print("Vehicles - get")
        pass
    
vehicle = Vehicles()
vehicle.get()

__init__
__get__
__call__
Accout Active - apply
Vehicles - get


In [39]:
# Using Class Descriptor with parameters

class AccountActive():
    def __init__(self, arg):
        print("__init__ , arg =", arg)
        self.arg = arg
        
    def __call__(self, func):
        print("__call__")
        self.func = func
        
        return self
    
    def __get__(self, instance, owner = None):
        print("__get__")
        if instance is None:
            return self
        
        def decorator(self_, *args, **kwargs):
            print("decorator")
            return self.func(self_, self, *args, **kwargs)
        
        return decorator.__get__(instance, owner)
    
    def apply(self):
        print("Accout Active - apply")
        

class Vehicles():
    def __init__(self):
        self.type = "car"
    
    @AccountActive("test")
    def get(self, decorator_instance):
        print("Vehicles - get")
        decorator_instance.apply()
    
vehicle = Vehicles()
vehicle.get()

__init__ , arg = test
__call__
__get__
decorator
Vehicles - get
Accout Active - apply


In [42]:
# Using Class Descriptor with parameters and a class attribute

class AccountActive():
    funcs = []
    
    def __init__(self, arg):
        print("__init__ , arg =", arg)
        self.arg = arg
        self.funcs.append(arg)
        
    def __call__(self, func):
        print("__call__")
        self.func = func
        
        return self
    
    def __get__(self, instance, owner = None):
        print("__get__")
        if instance is None:
            return self
        
        def decorator(self_, *args, **kwargs):
            print("decorator")
            return self.func(self_, self, *args, **kwargs)
        
        return decorator.__get__(instance, owner)
    
    def apply(self):
        print("Accout Active - apply, arg:", self.arg , "funcs:", self.funcs)
        

class Vehicles():
    def __init__(self):
        self.type = "car"
    
    @AccountActive("test")
    def get(self, decorator_instance):
        print("Vehicles - get")
        decorator_instance.apply()
        
    @AccountActive("test2")
    def get2(self, decorator_instance):
        print("Vehicles - get2")
        decorator_instance.apply()
    
vehicle = Vehicles()
vehicle.get()
vehicle.get2()

__init__ , arg = test
__call__
__init__ , arg = test2
__call__
__get__
decorator
Vehicles - get
Accout Active - apply, arg: test funcs: ['test', 'test2']
__get__
decorator
Vehicles - get2
Accout Active - apply, arg: test2 funcs: ['test', 'test2']


In [49]:
from functools import wraps

# Using Class Descriptor with parameters and appending wrapped function to funcs
# issue this is only appended to the func dict, once the wrapped function is called

class AccountActive():
    funcs = {}
    
    def __init__(self, arg):
        print("__init__ , arg =", arg)
        self.arg = arg
        
    def __call__(self, func):
        print("__call__")
        self.func = func
        return self
    
    def __get__(self, instance, owner = None):
        print("__get__")
        if instance is None:
            return self
        
        def decorator(self_, *args, **kwargs):
            print("decorator")
            
            
            @wraps(self.func)
            def wrapper(*args, **kwargs):
                return self.func(self_, self, *args, **kwargs)
            
            self.funcs[self.arg] = wrapper
            
            return wrapper(*args, **kwargs)
        return decorator.__get__(instance, owner)
    
    def apply(self):
        print("Accout Active - apply, arg:", self.arg , "funcs:", self.funcs)
        

class Vehicles():
    def __init__(self):
        self.type = "car"
    
    @AccountActive("test")
    def get(self, decorator_instance):
        print("Vehicles - get")
        decorator_instance.apply()
        
    @AccountActive("test2")
    def get2(self, decorator_instance):
        print("Vehicles - get2")
        decorator_instance.apply()
    
print("calling get")
vehicle = Vehicles()
vehicle.get()
vehicle.get2()

__init__ , arg = test
__call__
__init__ , arg = test2
__call__
calling get
__get__
decorator
Vehicles - get
Accout Active - apply, arg: test funcs: {'test': <function Vehicles.get at 0x00000210E33A5000>}
__get__
decorator
Vehicles - get2
Accout Active - apply, arg: test2 funcs: {'test': <function Vehicles.get at 0x00000210E33A5000>, 'test2': <function Vehicles.get2 at 0x00000210E33A5120>}


In [2]:
# With function wrapper and without singleton:
from functools import wraps

class AccountActive():
    funcs = {}
    
    def __init__(self, path):
        print("__init__: path =", path)
        self.path = path
        self.test = "Das ist ein Test"
    
    def __call__(self, func):
        print("__call__:", func)
        
        @wraps(func)
        def wrapper(instance, *args, **kwargs):       
            print("wrapper: instance =", instance, "args =", args)
            result = func(instance, self, *args, **kwargs)                
            return result
    
        @wraps(wrapper)
        def handler(*args, **kwargs):
            print("handler")
            return wrapper(instance, *args, **kwargs)
        
        print("add handler to funcs")
        self.funcs[self.path]= handler
        
        return wrapper
    
    def decorator_method(self, test):
        print("Execute decorator method: ", self.test, "parameter:", test)
                
class Vehicles():
    def __init__(self):
        self.type = "car"
    
    @AccountActive("/ws1")
    def get1(self, decorator, type):
        print("funcs:", decorator.funcs["/ws2"]("test"))
        print("Vehicles - get1: ", type)
        decorator.decorator_method("new param1")
        
    @AccountActive("/ws2")    
    def get2(self, decorator, type):
        print("Vehicles - get2: ", type)
        decorator.decorator_method("new param2")
    
    @AccountActive("/ws3")
    def get3(self, decorator, type):
        print("Vehicles - get3: ", type)
        decorator.decorator_method("new param3")
        
print("call methods")

vehicle = Vehicles()
vehicle.get1("vw")
vehicle.get2("bmw")
vehicle.get3("audi")


__init__: path = /ws1
__call__: <function Vehicles.get1 at 0x00000220125048B0>
add handler to funcs
__init__: path = /ws2
__call__: <function Vehicles.get2 at 0x0000022012504A60>
add handler to funcs
__init__: path = /ws3
__call__: <function Vehicles.get3 at 0x0000022012504C10>
add handler to funcs
call methods
wrapper: instance = <__main__.Vehicles object at 0x00000220124BAA40> args = ('vw',)
handler
wrapper: instance = <__main__.AccountActive object at 0x00000220124BBC70> args = ('test',)
Vehicles - get2:  test
Execute decorator method:  Das ist ein Test parameter: new param2
funcs: None
Vehicles - get1:  vw
Execute decorator method:  Das ist ein Test parameter: new param1
wrapper: instance = <__main__.Vehicles object at 0x00000220124BAA40> args = ('bmw',)
Vehicles - get2:  bmw
Execute decorator method:  Das ist ein Test parameter: new param2
wrapper: instance = <__main__.Vehicles object at 0x00000220124BAA40> args = ('audi',)
Vehicles - get3:  audi
Execute decorator method:  Das ist

: 